In [1]:
from glob import glob
from glob import escape
import pandas as pd

In [2]:
data_dir = '../datasets/amazon/*/'

In [5]:
user = glob(data_dir)[0]

In [9]:
fname_known = glob(escape(user) + '/*')[0]
fname_unknown = glob(escape(user) + '/*')[4]
print(fname_known)
print(fname_unknown)

../datasets/amazon/A1HGXLP6WATS4B[Y]/known - A1HGXLP6WATS4B - HomeandKitchen.txt
../datasets/amazon/A1HGXLP6WATS4B[Y]/unknown - A1HGXLP6WATS4B - OfficeProducts.txt


In [14]:
fname_known.split('/')[4].split(' - ')[1]

'A1HGXLP6WATS4B'

In [15]:
fname_unknown.split('/')[4].split(' - ')[1]

'A1HGXLP6WATS4B'

In [16]:
user_ids = []
text_ids = []
texts = []

for prob_dir in glob(data_dir):
    
    text_counter = 0
    files = glob(escape(prob_dir) + '/known*')

    for file in files:
        user_id = file.split('/')[4].split(' - ')[1]
        user_ids.append(user_id)
        text_ids.append(text_counter)
        text_counter += 1 
        fp = open(file)
        texts.append(fp.read())
        fp.close()

# put into dataframe
text_data = pd.DataFrame({'user_id': user_ids, 'text_id': text_ids, 'text': texts})

In [17]:
text_data

,user_id,text_id,text
0,A1HGXLP6WATS4B,0,"My decision to purchase a bagless upright, the..."
1,A1HGXLP6WATS4B,1,"After trying it once, I keep my pantry stocked..."
2,A1HGXLP6WATS4B,2,I was pleasantly surprised when I began using ...
3,A1HGXLP6WATS4B,3,This is a replacement for the original Trackma...
4,A226GFM0Q9L37H,0,This is a great formula for premature babies. ...
...,...,...,...
18301,AVFJ327UXPXLF,1,I purchased this movie because I felt that the...
18302,AVFJ327UXPXLF,2,"Nearly 30 years ago, I lived in Germany where ..."
18303,AVFJ327UXPXLF,3,I love these Builder Bars. They are the best t...
18304,AVFJ327UXPXLF,4,I bought this for weight lifting and running. ...


In [19]:
users = text_data.user_id.unique()

In [27]:
import json

In [31]:
from math import floor

In [32]:
floor(0.9*len(users))

2905

In [36]:
n_train_users = floor(0.9*len(users))
train_users = users[0:n_train_users]
test_users = users[n_train_users:len(users)]

In [38]:
test_user = train_users[0]

In [41]:
len(text_data.text[text_data.user_id == test_user].to_list())

4

In [42]:
import itertools

In [45]:
for combo in itertools.combinations(text_data.text[text_data.user_id == test_user].to_list(), 2):
    print(list(combo))

['My decision to purchase a bagless upright, the Bissell Cleanview, was made because I could no longer find bags locally for my other upright.The Bissell does a great job vacuuming carpets and picking up pet hair. I have no complaints about the performance. It has excellent suction and gets the deep down dirt and dust. The dirt cup is easy to remove, empty, and replace. The extension hose is strechier and of better quality than most of the competition in this price range.It doesn\'t have features such as a dirt-sensor, an on-off switch for the revolving brushes, or a floating cleaning head (it uses a manual height adjustment knob). What it lacks in features it makes up for by it\'s low purchase price and good performance.Why only 4 stars? It does have a few draw-backs of which some are inherent to a bagless vac.*Although the manual says monthly, the foam filters really need to be cleaned after each use. By the time you remove them, wash them, and reinstall them, it would have been a lo

In [46]:
import random

In [51]:
test_poss = itertools.combinations(test_users, 2)
test_pairs = random.sample(list(test_poss), 5)
for pair in test_pairs:
    print(pair[0])

A2RL1RS6PWUHWI
A3VONQL7YW8UPM
A1D1IDI5S9TN9D
A1D1IDI5S9TN9D
A3F3B1JPACN215


In [52]:
def create_json(users, split = "train"):
    fname = "../datasets/processed_amazon/amazon_" + split + ".jsonl"
    
    data_dict_list = []

    problem_id = 0

    # same source pairs
    for user in users:
        user_texts = text_data.text[text_data.user_id == user].to_list()
        for combo in itertools.combinations(user_texts, 2):
            data_dict = {
                'id': str(problem_id),
                'same': True,
                'authors': [user, user],
                'pair': list(combo)
            }
            problem_id+=1
            data_dict_list.append(data_dict)
    
    n_ss = problem_id

    # diff source pairs
    poss_ds = list(itertools.combinations(users, 2))
    ds_pairs = random.sample(poss_ds, n_ss)
    for pair in ds_pairs:
        user0 = pair[0]
        user1 = pair[1]
        user0_text = random.sample(text_data.text[text_data.user_id == user0].to_list(), 1)
        user1_text = random.sample(text_data.text[text_data.user_id == user1].to_list(), 1)
        data_dict = {
            'id': str(problem_id),
            'same': False,
            'authors': [user0_text, user1_text],
            'pair': [user0, user1]
        }
        problem_id+=1
        data_dict_list.append(data_dict)

    # write to file
    with open(fname, 'w') as f:
        for data_dict in data_dict_list:
            json.dump(data_dict, f)
            f.write("\n")

In [53]:
create_json(train_users, "train")

In [54]:
create_json(test_users, "test")